In [4]:
import requests, pandas as pd, time
from newspaper import Article
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

BASE_URL = "https://www.hankyung.com/article/"
HEADERS = {
    "User-Agent": ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/127.0.0.0 Safari/537.36"),
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8",
    "Referer": "https://www.hankyung.com/",
}

def crawl_url(url):
    # 단일 URL 크롤링: requests로 가져와 newspaper 파싱 (403시 /amp 폴백)
    try:
        r = requests.get(url, headers=HEADERS, timeout=12)
        if r.status_code == 403:
            r = requests.get(url.rstrip("/") + "/amp", headers=HEADERS, timeout=12)
        if r.status_code in (403, 404):
            return None
        r.raise_for_status()

        art = Article(url, language='ko')
        art.set_html(r.text)
        art.parse()
        if not art.title or not art.text:
            return None
        return {"url": url, "title": art.title, "text": art.text, "publish_date": art.publish_date}
    except:
        return None


def crawl_i_articles(date_str, max_workers=31):
    """i 기사 크롤링 (0000~9999)"""
    urls = [f"{BASE_URL}{date_str}{num:04}i" for num in range(10000)]
    results = []

    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = [ex.submit(crawl_url, u) for u in urls]
        collected = 0
        with tqdm(total=len(futures), desc=f"I {date_str}") as pbar:
            for fut in as_completed(futures):
                res = fut.result()
                if res:
                    results.append(res)
                    collected += 1
                pbar.set_postfix_str(f"수집 {collected}") # 현재 수집 건수
                pbar.update(1)

                time.sleep(0.06)  # 너무 빠른 요청 방지(차단 완화)

    df = pd.DataFrame(results).drop_duplicates(subset=["url"])
    tqdm.write(f"최종 수집 기사 수: {len(df)}") # 최종 수집 건 수
    return df

In [5]:
day_str = "20250822"

In [6]:
out_df = crawl_i_articles(day_str)
out_df.to_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_i_{day_str}.csv", index=False)

out_df

I 20250822: 100%|██████████| 10000/10000 [10:51<00:00, 15.35it/s, 수집 44]


최종 수집 기사 수: 44


,url,title,text,publish_date
0,https://www.hankyung.com/article/202508224739i,"""3년 반 만의 최고가""…인도, 결국 '이것' 수입 [원자재 포커스]",인도가 5년 만에 처음으로 카놀라유 수입에 나섰다. 현지 카놀라유(유채유) 가격이 ...,2025-08-22 08:02:36+09:00
1,https://www.hankyung.com/article/202508224815i,"[마켓PRO] Today's Pick : ""두산테스나, 계단식 성장""",두산테스나 서안성사업장 전경. 사진=두산테스나 제공\n\n두산테스나 - 계단식 성장...,2025-08-22 08:31:01+09:00
2,https://www.hankyung.com/article/202508224871i,[분석+]주춤한 치과 임플란트 3사...영업이익 전년 대비 반토막,"오스템임플란트, 중국서 ‘짐비’ 독점 유통\n\n임플란트 판매량 세계 1위 기업 오...",2025-08-22 10:30:00+09:00
3,https://www.hankyung.com/article/202508224911i,"라온시큐어, 양자내성암호 적용한 '터치엔 엠트랜스키' GS인증 1등급 획득","사진=라온시큐어\n\nIT 보안 인증 플랫폼 기업 ‘ 라온시큐어 ’(대표 이순형, ...",2025-08-22 08:58:59+09:00
4,https://www.hankyung.com/article/202508224973i,"""1초도 아까운데""…증권사 앱 사용자 30% ""오류·접속 장애 겪은 적 있어""",투자자 59% “앱 이용 불만”\n\n시스템 오류·접속 장애 가장 많아\n\n앱 만...,2025-08-22 13:39:35+09:00
5,https://www.hankyung.com/article/202508224974i,"'세관 마약 밀수 의혹' 합동수사팀, 동부지검으로 소속 변경",사진=뉴스1\n\n인천세관 공무원들의 마약 밀수 연루 및 수사 외압 의혹을 수사 중...,2025-08-22 09:24:36+09:00
6,https://www.hankyung.com/article/202508224982i,"[단독] 김상욱, 현직 의원 신분으로 대부업체 사내 이사 겸직","현직 의원 겸직 의무 위반 논란\n\n金 ""이사 등기 사실 몰랐다"" 해명\n\n김상...",2025-08-22 10:39:18+09:00
7,https://www.hankyung.com/article/202508224995i,"메디팜, '듀얼 프리미엄 초임계 rTG 오메가3' 출시","조아제약, 어린이 영양제 할랄 인증 획득\n\n조아제약은 인도네시아 할랄 인증기관 ...",2025-08-22 09:21:19+09:00
8,https://www.hankyung.com/article/202508225014i,"동아에스티, 그래디언트바이오컨버전스와 AI 기반 신약개발 협약","동아에스티, UNGC 가입…지속가능경영 국제기준 이행한다\n\n동아에스티가 유엔글로...",2025-08-22 09:26:02+09:00
9,https://www.hankyung.com/article/202508225053i,"아이엠바이오로직스, 中 진퀀텀과 ‘멀티 결합 항체-ADC’ 공동 연구",기존 IgG-ADC 한계를 극복\n\n신규 모달리티로 개발 가능\n\n아이엠바이오로...,2025-08-22 10:29:03+09:00
